In [9]:
import requests
from bs4 import BeautifulSoup

#URL страницы
url = "https://markets.businessinsider.com/index/components/s&p_500"

#отправка HTTP-запроса
response = requests.get(url)

#проверка успешности запроса
if response.status_code == 200:
    #создание объекта BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    
    #извлечение таблицы
    table = soup.find('table', class_='table__layout--fixed')
    
    #список для хранения ссылок
    full_links = []
    
    #извлечение всех ссылок из таблицы
    if table:
        for row in table.tbody.find_all('tr'):
            cols = row.find_all('td')
            if cols:
                link = cols[0].find('a')
                if link and 'href' in link.attrs:
                    href = link['href']
                    full_link = f"https://markets.businessinsider.com{href}"
                    full_links.append(full_link)

    #перебор всех ссылок и извлечение названия компании и тикера
    for link in full_links:
        
        #отправка запроса на страницу компании
        company_response = requests.get(link)
        
        if company_response.status_code == 200:
            company_soup = BeautifulSoup(company_response.text, 'html.parser')
            
            #поиск элемента с названием компании
            name_element = company_soup.find('span', class_='price-section__label')
            if name_element:
                company_name = name_element.get_text(strip=True)
            else:
                company_name = "N/A"

            #поиск элемента с тикером
            ticker_element = company_soup.find('span', class_='price-section__category')
            if ticker_element:
                ticker_text = ticker_element.get_text(strip=True)
            else:
                ticker_text = "N/A"
            
           # Поиск всех элементов с классом snapshot__data-item
            snapshot_items = company_soup.find_all('div', class_='snapshot__data-item')
            pe_value = "N/A"
            bid_value = "N/A"

            for item in snapshot_items:
                header = item.find('div', class_='snapshot__header')
                if header:
                    header_text = header.get_text(strip=True)
                    if header_text == "Bid":
                        bid_value = item.contents[0].strip()
                    elif header_text == "P/E Ratio":
                        pe_value = item.contents[0].strip()

            #поиск элемента с growth 
            growth_element = company_soup.find('td', class_='table__td text-right')
            if growth_element:
                growth_value = growth_element.get_text(strip=True)
            else:
                growth_value = "N/A"

            # Вывод данных
            print({
                "code": ticker_text,
                "name": company_name,
                "price": bid_value,
                "P/E": pe_value,
                "growth": growth_value
            })           
else:
    print(f'Failed: {response.status_code}')

{'code': 'Stock, MMM', 'name': '3M Co.', 'price': '150.32', 'P/E': '17.09', 'growth': '23,795'}
{'code': 'Stock, AOS', 'name': 'A.O. Smith Corp.', 'price': '66.75', 'P/E': '18.80', 'growth': '3,835'}
{'code': 'Stock, ABT', 'name': 'Abbott Laboratories', 'price': '126.61', 'P/E': '14.81', 'growth': '44,337'}
{'code': 'Stock, ABBV', 'name': 'AbbVie Inc', 'price': '211.58', 'P/E': '74.34', 'growth': '59,484'}
{'code': 'Stock, ACN', 'name': 'Accenture plc', 'price': '318.26', 'P/E': '29.90', 'growth': '68,624'}
{'code': 'Stock, ADBE', 'name': 'Adobe Inc.', 'price': '394.30', 'P/E': '41.73', 'growth': '23,492'}
{'code': 'Stock, AES', 'name': 'AES Corp.', 'price': '12.30', 'P/E': '5.44', 'growth': '12,349'}
{'code': 'Stock, AFL', 'name': 'Aflac Inc', 'price': '107.01', 'P/E': '10.74', 'growth': '17,248'}
{'code': 'Stock, A', 'name': 'Agilent Technologies Inc.', 'price': '120.84', 'P/E': '29.42', 'growth': '6,732'}
{'code': 'Stock, APD', 'name': 'Air Products and Chemicals Inc.', 'price': '29